# Effictient Yelp API Calls (CORE)

**Marco Jimenez 6/3/2022**

In [59]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [60]:
# Loading API Credentials
with open('/Users/LP-Ca/.secret/yelp_api.json') as f:
    creds = json.load(f)
creds.keys()

dict_keys(['client id', 'API key'])

In [61]:
# Instantiating YelpAPI Variable
yelp_api = YelpAPI(creds['API key'], timeout_s=5.0)
yelp_api

In [62]:
# Defining variables for search conditions
LOCATION = 'Pawcatuck, CT,06379'
TERM = 'Pizza'

**Creating results-in-progress JSON file**

In [63]:
JSON_FILE = f"Data/results_in_progress_Pizza.json"
JSON_FILE

'Data/results_in_progress_Pizza.json'

In [64]:
# Checking if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
# For loop in case it does not exist
if file_exists == False:
    # Getting folder name
    folder = os.path.dirname(JSON_FILE)
    # If JSON_FILE is included in folder:
    if len(folder)>0:
        # Create folder
        os.makedirs(folder, exist_ok=True)
    print(f"[i] {JSON_FILE} not found. Saving empty list to file")       
        
    # Saving first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Pizza.json already exists.


In [65]:
# Loading previous results and using len of results for offset
with open(JSON_FILE,'r') as f:
    prev_results = json.load(f)
    
## set offset based on previous results
n_results = len(prev_results)
print(f'- {n_results} previous results found.')

- 40 previous results found.


**Figuring out required number of pages**

In [66]:
# Performing API call and displayin dict keys
results = yelp_api.search_query(location=LOCATION,
                               term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [67]:
# Getting total number of results
total_results = results['total']
total_results

35

In [75]:
# Getting number of results per page
results_per_page = len(results['businesses'])
results_per_page

0

In [70]:
# Getting the number of pages by subtracting n_offset from the total and dividing the remainder by the number of results per page 
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

ZeroDivisionError: division by zero

**Adding results to .json file**

In [71]:
prev_results.extend(results['businesses'])
with open(JSON_FILE, 'w') as f:
     json.dump(prev_results,f)

**For loop to call each page**

In [72]:
pip install tqdm


Note: you may need to restart the kernel to use updated packages.


In [73]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    # Reading in results in progress file and checking the length
    with open(JSON_FILE, 'r') as f:
        prev_results = json.load(f)
    # Saving number of results to use as offset
    n_results = len(prev_results)
    # Using n_results as the offset value 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    # Appending new results and saving to file
    prev_results.extend(results['businesses'])
    
# Displaying previous_results
    with open(JSON_FILE,'w') as f:
        json.dump(prev_results,f)

  0%|          | 0/2 [00:00<?, ?it/s]

In [74]:
# Loading 'results in progress' json file into a dataframe
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,Zyfo5HuK1i1YZCv3U-GvwQ,the-pizza-lady-pawcatuck,The Pizza Lady,https://s3-media1.fl.yelpcdn.com/bphoto/16ZFRx...,False,https://www.yelp.com/biz/the-pizza-lady-pawcat...,37,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.379569, 'longitude': -71.836204}",[delivery],$,"{'address1': '50 Liberty St', 'address2': None...",+18605991113,(860) 599-1113,1169.704382
1,kNOjDM10zI0RvNEmThKmrA,pizzaplace-westerly-2,PizzaPlace,https://s3-media2.fl.yelpcdn.com/bphoto/S6ByHQ...,False,https://www.yelp.com/biz/pizzaplace-westerly-2...,121,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 41.376953, 'longitude': -71.830095}",[delivery],$$,"{'address1': '43 Broad St', 'address2': '', 'a...",+14013481803,(401) 348-1803,1657.997341
2,EDSQSFjQrrD56led7AvTpQ,mr-pizza-westerly,Mr. Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/_-FNJL...,False,https://www.yelp.com/biz/mr-pizza-westerly?adj...,18,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 41.36617, 'longitude': -71.830403...",[delivery],NaN,"{'address1': '49 Beach St', 'address2': None, ...",+14015967698,(401) 596-7698,2045.493602
3,5QQdidOV8ye4eZksu3k45A,nanas-bakery-and-pizza-westerly,Nana's Bakery & Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/pq8-t7...,False,https://www.yelp.com/biz/nanas-bakery-and-pizz...,7,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",5.0,"{'latitude': 41.37971, 'longitude': -71.82931}",[],NaN,"{'address1': '82 High St Grd Flr', 'address2':...",+14012133911,(401) 213-3911,1733.837858
4,zgNte7hf2UZsXpTBIoJkJQ,casa-della-luce-westerly,Casa Della Luce,https://s3-media1.fl.yelpcdn.com/bphoto/oAIn8p...,False,https://www.yelp.com/biz/casa-della-luce-weste...,143,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 41.360386, 'longitude': -71.811803}",[delivery],$$,"{'address1': '105 Franklin St', 'address2': ''...",+14016374575,(401) 637-4575,3694.928666


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
35,K-J1GXPvYtLMZDhT97eQBg,longo-ristorante-and-pizzeria-westerly,Longo Ristorante & Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/taf49q...,False,https://www.yelp.com/biz/longo-ristorante-and-...,56,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 41.379771, 'longitude': -71.829123}",[delivery],$$,"{'address1': '84 High St', 'address2': '', 'ad...",+14013150881,(401) 315-0881,1751.562612
36,HB6DPAR6s7QA0a0ZDFk8Fw,ritaccos-market-westerly,Ritacco's Market,https://s3-media3.fl.yelpcdn.com/bphoto/Psznan...,False,https://www.yelp.com/biz/ritaccos-market-weste...,7,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",5.0,"{'latitude': 41.383861, 'longitude': -71.815867}",[delivery],$$,"{'address1': '84 Oak St', 'address2': '', 'add...",+14015961835,(401) 596-1835,2932.566994
37,_iYvpGfCwqMa3iMMIypT1Q,supreme-cheese-pizzeria-and-ristorante-westerly-2,Supreme Cheese Pizzeria & Ristorante,https://s3-media4.fl.yelpcdn.com/bphoto/22yh-8...,False,https://www.yelp.com/biz/supreme-cheese-pizzer...,44,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 41.36834, 'longitude': -71.81974}",[delivery],$$,"{'address1': '157 Granite St', 'address2': '',...",+14015961001,(401) 596-1001,2702.987120
38,WIW3FD4EJDFFe3Qkfde6KQ,hilltop-cafe-westerly-3,Hilltop Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/Ph-g-a...,False,https://www.yelp.com/biz/hilltop-cafe-westerly...,16,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 41.38136394732243, 'longitude': -...",[delivery],$$,"{'address1': '23 Canal St', 'address2': '', 'a...",+18309978922,(830) 997-8922,1574.115634
39,Lx6EeL5mwVtNdBf9uUEVQA,dominos-pizza-westerly,Domino's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/2UoGpR...,False,https://www.yelp.com/biz/dominos-pizza-westerl...,10,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 41.37312, 'longitude': -71.8207}",[],$,"{'address1': '93 Granite St', 'address2': '', ...",+14015968850,(401) 596-8850,2466.757527
